### Modeling DAU with Markov Chain

Doubtlessly, **DAU**, **WAU**, and **MAU** — daily, weekly, and monthly active users — are critical business metrics. Cost estimations often require DAU forecasts.

Prediction using Duolingo’s growth model - this approach is better compared to standard time-series forecasting methods, and can be adjusted the prediction according to your teams' plans (e.g., marketing, activation, product teams).

#### Methodology

A quick recap on how the [Duolingo’s growth model](https://blog.duolingo.com/growth-model-duolingo/) works. At day d (d = 1, 2, … ) of a user’s lifetime, the user can be in one of the following 7 (mutually-exclusive) states: `new`, `current`, `reactivated`, `resurrected`, `at_risk_wau`, `at_risk_mau`, `dormant`. The states are defined according to indicators of whether a user was active today, in the last 7 days, or in the last 30 days.

Having these states defined (as a set `S`), we can consider user behavior as a `Markov chain`. Here’s an example of a user’s trajectory: `new→ current→ current→ at_risk_wau→...→ at_risk_mau→...→ dormant`. Let M be a transition matrix associated with this Markov process: `m_{i, j} = P(s_j | s_i)` are the probabilities that a user moves to state `s_j` right after being at state `s_i`, where `s_i, s_j ∈ S`. Such a matrix is inferred from the historical data.